# Data Dictionary Workbook

This notebook contains tools to transform metadata among data dictionary formats.

A **Data Dictionary** is a database that stores information about data.   This information, called **Metadata**, includes the name of the data, type of data, the location of the data, and the meaning of the data.  

Metadata is not only useful as documentation for understanding data.   Metadata queried from a catalog is also useful as input parameters for process automation.  Metadata-driven automation enables common processes to be generally applied to the various and sundry datasets described in the data dictionary.  Metadata is the key to implementing an Enterprise approach to process automation.

The Bureau is currently utilizing, or planning to utilize, several data dictionary formats:
- **AWS Glue Catalog** stores structural metadata about data assets hosted on AWS Native Services
- **Data Asset Structure Metadata** (attached 'dataAssetMetadata.csv', ref Ch 3 of OCDO Playbook Fig. 3.2)
- **FiscalData** Data Dictionaries (csv) and Meta Ojbects (json) embedded in API result sets to  publically-accessible Bureau information.
- **RAML** is a modeling language for REST APIs.  It is used to decribe the functionality and data content of the API.   Mulesoft, at the core of the Fiscal Service Enterprise API Strategy, utilizes RAML to define APIs published on Exchange.
- **Unity Catalog** (future) stores structural metadata about data assets hosted on the Databricks platform (USA Spending, FS Data Hub).


## FiscalData Data Dictionaries
The Department of the Treasury and the Bureau of the Fiscal Service created FiscalData to consolidate federal financial data into one easy-to-use website. 
https://fiscaldata.treasury.gov/

FiscalData datasets are available for download in multiple machine-readable formats and feature customizable data previews. Each dataset is complete with comprehensive metadata to explain when the data is updated, what each row of the data represents, and descriptions for each dataset and data table.

Structural metadata is provided as 
1. Data Dictionaries downloadable as CSV files
2. Meta objects embedded in API responses

The following Python functions transform each into RAML Data Types

In [10]:
def export_fddatadict_to_raml( csv_input_path, raml_output_path='' ):
    ''' export FiscalData Data Dictionary CSV file to RAML v1.0 DataType '''
    import csv
    import json
    import yaml

    #data = {}
    raml_types = {}

    # Open a csv reader called DictReader
    with open(csv_input_path, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)

        # Convert each row into a dictionary and add it to RAML output
        for row in csvReader:
            dataset = row['dataset']
            data_table_name = row['data_table_name']
            field_name = row['field_name']

            # convert dictionary data type to RAML data type ...
            data_type = row['data_type'].lower()
            if data_type == 'date':
                data_type = 'datetime'
            elif data_type in ('currency','year','quarter','month','day','percentage'):
                # ... for simplicity -- RAML does support 'custom' object types that can define discrete charactistics
                data_type = 'number'
            else:
                data_type = 'string'

            if data_table_name not in raml_types.keys():
                raml_types[data_table_name] = {
                    'type' : 'object',
                    'properties' : {} }
            raml_types[data_table_name]['properties'][field_name] = {
                'type' : data_type,
                'displayName' : row['display_name'],
                'description' : row['description'],
                'required' : row['is_required'] == '1'
            }
                
    yaml_template = f"#%RAML 1.0\n\ntitle: Import from FiscalData Data Dictionary {dataset} Dataset\n"
    raml_types = {"types" : raml_types}
    raml = yaml_template + yaml.dump(raml_types, sort_keys=False)

    if raml_output_path > '':
        with open( raml_output_path, 'w') as f:
            resp = f.write(raml)
        f.close()
        print( f"Output to file '{raml_output_path}'" )
    else:
        print( raml)

    return 


In [11]:
# example from https://fiscaldata.treasury.gov/datasets/treasury-offset-program/federal-collections 'Download Data Dictionary' link ...
csv_input_path = "Treasury Offset Program Data Dictionary.csv"
raml_output_path = csv_input_path.replace('.csv','.raml')

export_fddatadict_to_raml( csv_input_path, raml_output_path )

Output to file 'Treasury Offset Program Data Dictionary.raml'


In [3]:
def export_fdmeta_to_raml( json_meta, raml_output_path='' ):
    ''' export FiscalData Meta Object (from API Response) to RAML v1.0 DataType '''

    import yaml

    raml_types = {}

    for key in meta['labels'].keys():
        #print(key)
        raml_types[key] = {
                'type' : meta['dataTypes'][key].lower() ,
                'displayName' : meta['labels'][key] ,
                'format' : meta['dataFormats'][key]
            }

    yaml_template = f"#%RAML 1.0\n\ntitle: Import from FiscalData Meta Object\n"
    raml_types = {"types" : raml_types}
    raml = yaml_template + yaml.dump(raml_types, sort_keys=False)

    if raml_output_path > '':
        with open( raml_output_path, 'w') as f:
            resp = f.write(raml)
        f.close()
        print( f"Output to file '{raml_output_path}'" )
    else:
        print( raml)

    return

In [4]:
# example from https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/debt/top/top_federal?fields=record_date,creditor_agency_nm,creditor_agency_id,agency_type,agency_site
# meta object copied from response ...
meta = {
  "count": 100,
  "labels": {
    "record_date": "Record Date",
    "creditor_agency_nm": "Creditor Agency Name",
    "creditor_agency_id": "Agency ID",
    "agency_type": "Agency Site Type Code",
    "agency_site": "Agency Site ID"
  },
  "dataTypes": {
    "record_date": "DATE",
    "creditor_agency_nm": "STRING",
    "creditor_agency_id": "STRING",
    "agency_type": "STRING",
    "agency_site": "STRING"
  },
  "dataFormats": {
    "record_date": "YYYY-MM-DD",
    "creditor_agency_nm": "String",
    "creditor_agency_id": "String",
    "agency_type": "String",
    "agency_site": "String"
  },
  "total-count": 161141,
  "total-pages": 1612
}

export_fdmeta_to_raml( meta )

#%RAML 1.0

title: Import from FiscalData Meta Object
types:
  record_date:
    type: date
    displayName: Record Date
    format: YYYY-MM-DD
  creditor_agency_nm:
    type: string
    displayName: Creditor Agency Name
    format: String
  creditor_agency_id:
    type: string
    displayName: Agency ID
    format: String
  agency_type:
    type: string
    displayName: Agency Site Type Code
    format: String
  agency_site:
    type: string
    displayName: Agency Site ID
    format: String



In [ ]:
def url_to_sql( url_input ):

    from urllib.parse import urlparse
    parts = urlparse(url_input)

    parsed_url = {
        'UrlInput' : url_input,
        'BaseUrl' : f"{parts.scheme}://{parts.netloc}",
        'Endpoint' : parts.path,
        'TableName' : parts.path.split('/')[-1],
        'ColumnList' : '*',
        'FilterSpec' : '',
        'SortSpec' : '',
        'PageSize' : 100,
        'PageNumber' : 1,
        'Offset' : 0,
        'Format' : 'json' # or 'csv'
    }

    #if url_input.__contains__('?'):
    if parts.query > '':
        url_parms = parts.query.replace(' ','').split('&')
        for parm in url_parms:
            parm_value = parm.split('=')[1]
            if parm.startswith('fields='):
                parsed_url['ColumnList'] = parm_value
            elif parm.startswith('filter='):
                filters = parm_value
                operators = {
                    ':lt:'  : ' < ',   # Less than
                    ':lte:' : ' <= ',  # Less than or equal to
                    ':gt:'  : ' > ',   # Greater than
                    ':gte:' : ' >= ',  # Greater than or equal to
                    ':eq:'  : ' = ',   # Equal to
                    ':in:'  : ' in '   # Contained in a given set
                    }
                '''
                if ':in:(' in filters:

                    filters[filters.index('(')+1 : filters.index(')')]

                    in_values = filters[filters.find(':in:(')+5:filters.find(')')].split(',')
                    for i, val in enumerate(in_values):in_values[i] = f"'{val}'"
                    in_values = ",".join(in_values)
                '''
                for op in operators:
                    #print ( op, operators[op] )

                    if op ==':in:':
                        lp = filters.find('(') + 1
                        rp = filters.find(')', lp)
                        in_values = filters[lp:rp]

                    filters = filters.replace(op, operators[op])
                parsed_url['FilterList'] = filters.split(',')
                #parsed_url['FilterList'] = parm_value

            elif parm.startswith('sort='):
                parsed_url['SortSpec'] = parm_value

            elif parm.startswith('format='):
                parsed_url['Format'] = parm_value

            elif parm.startswith('page[size]='):
                page_size = int(parm_value)
                parsed_url['PageSize'] = int(parm_value)

            elif parm.startswith('page[number]='):
                page_number = int(parm_value)
                parsed_url['PageNumber'] = page_number
                parsed_url['Offset'] = page_size * (page_number - 1)

            else:
                print( f'Ignoring unknown parm {parm}' )

        '''
          select {ColumnList}
            from {TableName}
           where {FilterSpec}
        order by {SortSpec}
           limit {PageSize} offset {PageSize * PageNumber} ;
        '''
        sql =  f"  select {'ColumnList'} \n    from {parsed_url['TableName']} \n"
        if parsed_url.['FilterList'] > '':
            sql += f"   where " + {' and '.join(parsed_url['FilterList'])} + " \n"
        if parsed_url.['SortSpec'] > '':
            sql += f"order by {parsed_url.['SortSpec']} \n"
        if parsed_url.['PaginationSpec'] > '':
            sql += f"  {parsed_url.['PaginationSpec']} \n"
        sql += ";"

        parsed_url['Sql'] = sql

    print (parsed_url)
    return parsed_url


In [ ]:
url_input = 'https://api.fiscaldata.treasury.gov:443/services/api/fiscal_service/v2/accounting/od/debt_to_penny?fields=record_calendar_year&sort=-record_calendar_year,-record_calendar_month'
url_input = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/rates_of_exchange?fields=country_currency_desc,exchange_rate, record_date&filter=country_currency_desc:in:('Canada-Dollar','Mexico-Peso'), record_date:gte:2020-01-01 &foobar=someshit"

url_input = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/{table_name}'


parts.query.strip('&')


parsed_url = url_to_sql(url_input)
','.join(parsed_url['ColumnList'])

'where ' + ' and '.join(parsed_url['FilterList'])


print(json.dumps(parsed_url, indent=2))
